In [1]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math

In [2]:
train_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_df.reset_index(inplace=True)
train_df.head()

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Store,Date,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-07-31,5,5263,555,1,1,0,1
1,2,2015-07-31,5,6064,625,1,1,0,1
2,3,2015-07-31,5,8314,821,1,1,0,1
3,4,2015-07-31,5,13995,1498,1,1,0,1
4,5,2015-07-31,5,4822,559,1,1,0,1


In [3]:
test_df = DataFrame.from_csv("test.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
test_df.reset_index(inplace=True)
test_df.head()

,Store,Date,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-09-17,1,4,1,1,0,0
1,3,2015-09-17,2,4,1,1,0,0
2,7,2015-09-17,3,4,1,1,0,0
3,8,2015-09-17,4,4,1,1,0,0
4,9,2015-09-17,5,4,1,1,0,0


# "Train" the model: Calculate mean per group over full training set

In [4]:
train_df = train_df[train_df['Sales'] > 0]
train_df['Open'] = train_df['Open'].fillna(1)


In [5]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Sales'].mean()
    means = means.reset_index()
    return means

means = calculate_means(train_df)
train_df = pandas.merge(train_df, means, on = ['Store','DayOfWeek','Promo'], how='left', suffixes=["", "_predicted"])
train_df['Sales_predicted'] = train_df['Sales_predicted'].fillna(train_df['Sales_predicted'].mean())


### Score the model

In [6]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [7]:
score_df = train_df[['Sales', 'Sales_predicted']].copy()
score_df = score_df[score_df['Sales'] != 0]
score_df['PercentError'] =  (score_df['Sales'] - score_df['Sales_predicted']) / score_df['Sales']
score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
math.sqrt(score_df['PercentErrorSquared'].mean())

0.22978844842293555

## Featurize and save

In [8]:
train_df[['Store', 'Date', 'Sales', 'Sales_predicted']].to_csv("train-features-predicted_mean.csv")

In [10]:

test_df = pandas.merge(test_df, means, on = ['Store','DayOfWeek','Promo'], how='left')

In [12]:
test_df['Sales_predicted'] = test_df['Sales']
del test_df['Sales']

In [13]:
test_df[['Store', 'Date', 'Sales_predicted']].to_csv("test-features-predicted_mean.csv")